In [96]:
# General imports
import numpy as np
import torch

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta
from sklearn.linear_model import ARDRegression

import seaborn as sns
sns.set()

# Making data

In [97]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)

y = dataset.time_deriv(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1)) # observations
X = dataset.library(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), poly_order=2, deriv_order=3) # covariates

print(y.shape, X.shape)

(5000, 1) (5000, 12)


In [98]:
y += np.std(y) * 0.5 * np.random.randn(*y.shape)

In [101]:
1 / (np.std(y) * 0.5)**2

124.31148290209636

In [100]:
X = X / np.linalg.norm(X, axis=0, keepdims=True)

# Baseline

In [102]:
reg = ARDRegression(alpha_1=0, alpha_2=0, lambda_1=0, lambda_2=0, fit_intercept=False, compute_score=True)

In [103]:
reg.fit(X, y)

/home/gert-jan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


ARDRegression(alpha_1=0, alpha_2=0, compute_score=True, fit_intercept=False,
              lambda_1=0, lambda_2=0)

In [104]:
reg.coef_[:, None]

array([[ 5.25206972e-02],
       [-4.94513305e-04],
       [ 7.43065421e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [-1.04260056e+01],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 1.22884590e-01]])

In [105]:
reg.lambda_

array([1.71445116e+02, 8.62351648e+03, 1.81090137e-02, 1.34023282e+04,
       1.40524600e+04, 9.19910893e-03, 2.81571225e+04, 9.97261754e+04,
       2.01255070e+04, 1.57324925e+04, 3.58915212e+04, 4.52129422e+01])

In [106]:
reg.alpha_

153.9288525299634

In [107]:
reg.scores_

[10039.051253594791,
 10089.093000438887,
 10091.168031144203,
 10104.031580754345,
 10107.943032728228,
 10112.089942735729,
 10112.071692677218,
 10116.573595707348,
 10116.574835473974,
 10116.575505725265,
 10116.57589785231,
 10116.576140932826,
 10116.576298073498,
 10116.576402733874,
 10116.57647391213,
 10116.576523026388,
 10116.576557257968]

# Own implementation

In [91]:
def SBL(X, y, alpha_, beta_):
    N, M = X.shape[0], X.shape[1]
    
    gram = X.T @ X
    sigma = torch.inverse(torch.diag(alpha_ / beta_) + gram)
    mn = sigma @ X.T @ y
    
    p = - N * torch.log(beta_) - torch.logdet(torch.eye(M) -  gram @ sigma) + beta_ * (y.T @ y - y.T @ X @ mn)
    return p, mn

In [92]:
# Now let's optimize
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

a = torch.nn.Parameter(torch.zeros(12, dtype=torch.float32))
b = torch.nn.Parameter(-torch.log(torch.var(y)))

optimizer = torch.optim.Adam([a, b], lr=1e-2)
max_epochs=1e4

<ipython-input-92-0abe2f548606>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
<ipython-input-92-0abe2f548606>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


In [93]:
for epoch in torch.arange(max_epochs):
    alpha_ = torch.exp(a).clamp(max=1e8)
    beta_ = torch.exp(b).clamp(max=1e8)
    loss = SBL(X, y, alpha_, beta_)[0]
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(loss)

tensor([[1085.0990]], grad_fn=<AddBackward0>)
tensor([[46.3650]], grad_fn=<AddBackward0>)
tensor([[33.3966]], grad_fn=<AddBackward0>)
tensor([[32.2879]], grad_fn=<AddBackward0>)
tensor([[32.2120]], grad_fn=<AddBackward0>)
tensor([[32.1989]], grad_fn=<AddBackward0>)
tensor([[32.1909]], grad_fn=<AddBackward0>)
tensor([[32.1859]], grad_fn=<AddBackward0>)
tensor([[32.1828]], grad_fn=<AddBackward0>)
tensor([[32.1809]], grad_fn=<AddBackward0>)


In [94]:
alpha_

tensor([2.7518e+03, 1.0846e+02, 2.0792e-02, 2.1820e+03, 2.7587e+03, 9.6296e-03,
        3.1648e+03, 5.0766e+03, 4.1529e+03, 4.0245e+01, 2.3645e+03, 2.9882e+03],
       grad_fn=<ClampBackward>)

In [95]:
1 / beta_

tensor(3.1971, grad_fn=<MulBackward0>)

In [82]:
mn

tensor([[ 3.2022e-04],
        [-9.9967e-04],
        [ 7.3045e+00],
        [-1.0570e-01],
        [ 1.7910e-02],
        [-1.0344e+01],
        [ 3.3136e-02],
        [ 3.8003e-02],
        [ 5.9642e-02],
        [-2.2907e-01],
        [ 1.4715e-02],
        [-3.1639e-02]], grad_fn=<MmBackward>)

In [83]:
mn * (alpha_ < 1e4)[:, None]

tensor([[ 3.2022e-04],
        [-0.0000e+00],
        [ 7.3045e+00],
        [-0.0000e+00],
        [ 0.0000e+00],
        [-1.0344e+01],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [-2.2907e-01],
        [ 0.0000e+00],
        [-0.0000e+00]], grad_fn=<MulBackward0>)